In [6]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score

In [7]:
df = pd.read_csv("E:/21KDT-Project-master/4. dms_seoul_avg.csv")
df.head()

,date,mosquito,temp,rain_per_day,accum_rain,wind,humidity,sunshine
0,2015-04-06,199,12.214286,5.285714,0,3.371429,68.757143,8.880000
1,2015-04-07,146,11.571429,4.928571,0,3.428571,65.742857,9.891429
2,2015-04-08,90,10.914286,4.571429,0,3.257143,63.542857,9.927143
3,2015-04-09,172,10.171429,0.571429,0,2.871429,59.614286,10.884286
4,2015-04-10,249,10.314286,0.071429,0,2.871429,53.485714,12.777143


In [8]:
y_target = df['mosquito'].values 
X_data = df.drop(['mosquito','date'], axis=1).values 

In [11]:
from sklearn.linear_model import Lasso, ElasticNet

def get_linear_reg_eval(model_name, params = None, X_data_n = None, y_target_n = None, verbose = True):
    coeff_df = pd.DataFrame()
    if verbose : 
        print('####### ', model_name , '#######')
    for param in params:
        if model_name == 'Ridge' : 
            model = Ridge(alpha = param)
        elif model_name == 'Lasso' :
            model = Lasso(alpha = param)
        elif model_name == 'ElasticNet' :
            model = ElasticNet(alpha = param, l1_ratio = 0.7)
        
        neg_mse_scores = cross_val_score(model, X_data_n, y_target_n, scoring = 'neg_mean_squared_error',cv=5)
        avg_rmse = np.mean(np.sqrt(-1 * neg_mse_scores))
        
        print('alpha {0} 일 때 5 folds 의 평균 RMSE : {1:.3f}'.format(param,avg_rmse))
        
        model.fit(X_data, y_target)
        coeff = pd.Series(data=model.coef_, index = ['temp', 'rain_per_day', 'accum_rain', 'wind', 'humidity', 'sunshine'])
        colname = 'alpha:' +str(param)
        coeff_df[colname] = coeff
    return coeff_df

In [12]:
# Lasso에 사용될 알파 파아미터의 값들을 정의하고 get_linear_reg_eval() 함수 호출
lasso_alphas = [0.07, 0.1, 0.5, 1, 3]
coeff_lasso_df = get_linear_reg_eval('Lasso', params=lasso_alphas, X_data_n=X_data, y_target_n=y_target)

#######  Lasso #######
alpha 0.07 일 때 5 folds 의 평균 RMSE : 2167.341
alpha 0.1 일 때 5 folds 의 평균 RMSE : 2167.342
alpha 0.5 일 때 5 folds 의 평균 RMSE : 2167.361
alpha 1 일 때 5 folds 의 평균 RMSE : 2167.386
alpha 3 일 때 5 folds 의 평균 RMSE : 2167.462


In [13]:
sort_column = 'alpha:'+str(lasso_alphas[0])
coeff_lasso_df.sort_values(by = sort_column, ascending = False)

,alpha:0.07,alpha:0.1,alpha:0.5,alpha:1,alpha:3
wind,920.988181,920.834527,918.785992,916.225266,905.982482
temp,290.581806,290.576152,290.500730,290.406462,290.029335
accum_rain,3.695724,3.695387,3.690898,3.685287,3.662843
humidity,3.096525,3.096740,3.099675,3.103324,3.117991
sunshine,-37.735593,-37.730805,-37.666950,-37.587132,-37.267835
rain_per_day,-72.073369,-72.069715,-72.021126,-71.960351,-71.717358
